In [1]:
from transformers import (
    AutoTokenizer,
    TFAutoModelForQuestionAnswering,
    DefaultDataCollator,
    keras_callbacks,
)
import tensorflow as tf
from huggingface_hub import notebook_login
from question_answering.constants import constants
from question_answering.utils import core_qa_utils, extractive_qa_utils
from question_answering.paths import extractive_qa_paths

In [2]:
df_train, df_val, df_test = core_qa_utils.load_train_val_test_datasets(
    extractive_qa_paths.squad_dataset_dir
)

train_dataset, val_dataset, test_dataset = core_qa_utils.convert_dataframes_to_datasets(
    [df_train, df_val, df_test]
)

In [3]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
def tokenize_sample(sample, max_tokens=None, padding=False):
    question = sample["question"].strip()
    context = sample["context"].strip()

    return tokenizer(question, context, max_length=max_tokens, padding=padding)


tokenized_train_dataset = train_dataset.map(tokenize_sample)
tokenized_val_dataset = val_dataset.map(tokenize_sample)
tokenized_test_dataset = test_dataset.map(tokenize_sample)

print(
    "Max number of tokens in tokenized train dataset: ",
    len(max(tokenized_train_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized val dataset: ",
    len(max(tokenized_val_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized test dataset: ",
    len(max(tokenized_test_dataset["input_ids"], key=len)),
)

Map:   0%|          | 0/68716 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/14724 [00:00<?, ? examples/s]

Map:   0%|          | 0/14725 [00:00<?, ? examples/s]

Max number of tokens in tokenized train dataset:  870
Max number of tokens in tokenized val dataset:  866
Max number of tokens in tokenized test dataset:  817


In [5]:
max_length = 384


def filter_samples_below_number_of_tokens(dataset, max_tokens: int):
    indices_to_remove = []

    # Find indices of samples where number of tokens exceeds max number of tokens
    for index, sample in enumerate(dataset):
        tokenized_sample = tokenize_sample(sample)
        if len(tokenized_sample["input_ids"]) > max_tokens:
            indices_to_remove.append(index)

    # Keep only samples with number of tokens less or equal than max number of tokens
    dataset_indices = range(len(dataset))
    filtered_dataset = dataset.select(
        index for index in dataset_indices if index not in set(indices_to_remove)
    )

    return filtered_dataset


filtered_train_dataset = filter_samples_below_number_of_tokens(
    train_dataset, max_tokens=max_length
)
filtered_val_dataset = filter_samples_below_number_of_tokens(
    val_dataset, max_tokens=max_length
)
filtered_test_dataset = filter_samples_below_number_of_tokens(
    test_dataset, max_tokens=max_length
)

Parameter 'indices'=<generator object filter_samples_below_number_of_tokens.<locals>.<genexpr> at 0x0000015357C2CEB0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [6]:
print(
    "Number of samples in tokenized train dataset before filtering: ",
    len(train_dataset),
)
print("Number of samples in tokenized val dataset before filtering: ", len(val_dataset))
print(
    "Number of samples in tokenized test dataset before filtering: ", len(test_dataset)
)

print("\n---------------\n")

print(
    "Number of samples in tokenized train dataset after filtering: ",
    len(filtered_train_dataset),
)
print(
    "Number of samples in tokenized val dataset after filtering: ",
    len(filtered_val_dataset),
)
print(
    "Number of samples in tokenized test dataset after filtering: ",
    len(filtered_test_dataset),
)

Number of samples in tokenized train dataset before filtering:  68716
Number of samples in tokenized val dataset before filtering:  14724
Number of samples in tokenized test dataset before filtering:  14725

---------------

Number of samples in tokenized train dataset after filtering:  67964
Number of samples in tokenized val dataset after filtering:  14573
Number of samples in tokenized test dataset after filtering:  14552


In [7]:
def preprocess_dataset(dataset):
    questions = [q.strip() for q in dataset["question"]]
    contexts = [c.strip() for c in dataset["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        padding="max_length",
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")

    answer_start_indices = dataset["answer_start"]
    answer_texts = dataset["answer_text"]
    start_positions = []
    end_positions = []

    for index, offset in enumerate(offset_mapping):
        start_char = answer_start_indices[index]
        end_char = start_char + len(answer_texts[index])
        sequence_ids = inputs.sequence_ids(index)

        # Find the start and end token indices of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
tokenized_train_dataset = filtered_train_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_train_dataset.column_names,
)
tokenized_val_dataset = filtered_val_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_val_dataset.column_names,
)
tokenized_test_dataset = filtered_test_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_test_dataset.column_names,
)

Map:   0%|          | 0/67964 [00:00<?, ? examples/s]

Map:   0%|          | 0/14573 [00:00<?, ? examples/s]

Map:   0%|          | 0/14552 [00:00<?, ? examples/s]

In [9]:
print(
    f"All tokenized train dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_train_dataset["input_ids"]
        ]
    ),
)
print(
    f"All tokenized val dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_val_dataset["input_ids"]
        ]
    ),
)
print(
    f"All tokenized test dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_test_dataset["input_ids"]
        ]
    ),
)

All tokenized train dataset entries have 384 tokens:  True
All tokenized val dataset entries have 384 tokens:  True
All tokenized test dataset entries have 384 tokens:  True


In [10]:
# Model parameters
training_number = 1

model_name = "squad-bert-uncased"
full_model_name = f"{model_name}-{training_number}"

# Checkpoints
checkpoint_filename_template = constants.checkpoint_filename_template
checkpoints_path = (
    extractive_qa_paths.training_checkpoints_dir
    / full_model_name
    / checkpoint_filename_template
)

# Hub
hub_path = extractive_qa_paths.hub_models_location / full_model_name

# Saved models
saved_models_path = extractive_qa_paths.saved_models_dir / full_model_name

# Figures
figures_dir = extractive_qa_paths.figures_dir / full_model_name

# Hyperparameters
batch_size = 8
train_epochs = 10

In [11]:
# Load model for fine-tuning
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Dataset preparation
data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_train_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["start_positions", "end_positions"],
    collator=data_collator,
    batch_size=batch_size,
)

tf_val_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_val_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["start_positions", "end_positions"],
    collator=data_collator,
    batch_size=batch_size,
)

tf_test_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_test_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["start_positions", "end_positions"],
    collator=data_collator,
    batch_size=batch_size,
)

In [13]:
# Login to hugging face hub in order to store the model there
# notebook_login()

In [14]:
# Callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoints_path, verbose=1, save_weights_only=True
)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=1)
# push_to_hub = keras_callbacks.PushToHubCallback(
#     output_dir=full_model_name, tokenizer=tokenizer
# )

callbacks = [
    checkpoint_cb, 
    early_stop_cb, 
    # push_to_hub
]

In [15]:
# Compile
num_train_steps = len(tf_train_dataset) * train_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=2e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Compile
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070, compute capability 8.9


In [16]:
model.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108891648 
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
Total params: 108,893,186
Trainable params: 108,893,186
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Fit the model on the new data
history = model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=train_epochs,
    callbacks=callbacks,
)

Epoch 1/10
8496/8496 [==============================] - ETA: 0s - loss: 2.6453 - end_logits_loss: 1.2818 - start_logits_loss: 1.3635 - end_logits_accuracy: 0.6536 - start_logits_accuracy: 0.6125
Epoch 1: saving model to e:\STUDIA\IPS\question-answering\extractive-qa\training-checkpoints\squad-bert-uncased-1\cp-01.ckpt
8496/8496 [==============================] - 2341s 273ms/step - loss: 2.6453 - end_logits_loss: 1.2818 - start_logits_loss: 1.3635 - end_logits_accuracy: 0.6536 - start_logits_accuracy: 0.6125 - val_loss: 1.9941 - val_end_logits_loss: 0.9562 - val_start_logits_loss: 1.0379 - val_end_logits_accuracy: 0.7353 - val_start_logits_accuracy: 0.6930
Epoch 2/10
8496/8496 [==============================] - ETA: 0s - loss: 1.5734 - end_logits_loss: 0.7526 - start_logits_loss: 0.8208 - end_logits_accuracy: 0.7785 - start_logits_accuracy: 0.7416
Epoch 2: saving model to e:\STUDIA\IPS\question-answering\extractive-qa\training-checkpoints\squad-bert-uncased-1\cp-02.ckpt
8496/8496 [=====

In [18]:
# Get best version of the model
best_model = core_qa_utils.get_best_model_from_checkpoints(
    model, history, model_name=full_model_name, remove_checkpoints=False
)

In [19]:
# Save best model's weights
extractive_qa_utils.save_model(best_model, model_name=full_model_name)

In [20]:
# Load best model
loaded_best_model = extractive_qa_utils.load_weights_into_model(
    best_model, model_name=full_model_name
)

In [21]:
# Get predictions from best_model
best_model_preds = best_model.evaluate(tf_test_dataset)

1819/1819 [==============================] - 262s 144ms/step - loss: 2.0131 - end_logits_loss: 0.9623 - start_logits_loss: 1.0508 - end_logits_accuracy: 0.7308 - start_logits_accuracy: 0.6905


In [22]:
loaded_best_model_preds = loaded_best_model.evaluate(tf_test_dataset)

1819/1819 [==============================] - 262s 144ms/step - loss: 2.0131 - end_logits_loss: 0.9623 - start_logits_loss: 1.0508 - end_logits_accuracy: 0.7308 - start_logits_accuracy: 0.6905


In [23]:
best_model_preds

[2.0130584239959717,
 0.9622660279273987,
 1.0507932901382446,
 0.7307586669921875,
 0.6904892921447754]

In [24]:
loaded_best_model_preds

[2.013058662414551,
 0.9622660279273987,
 1.0507932901382446,
 0.7307586669921875,
 0.6904892921447754]

In [27]:
modelsko = extractive_qa_utils.load_model(model_checkpoint=model_checkpoint, model_name=full_model_name)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
modelsko_best_model_preds = modelsko.predict(tf_test_dataset)

1819/1819 [==============================] - 98s 52ms/step


In [30]:
modelsko_best_model_preds

TFQuestionAnsweringModelOutput(loss=None, start_logits=array([[-6.855, -4.04 , -6.355, ..., -7.93 , -7.957, -7.984],
       [-7.074, -5.15 , -6.82 , ..., -7.934, -7.95 , -7.95 ],
       [-6.984, -4.03 , -5.293, ..., -7.81 , -7.805, -7.74 ],
       ...,
       [-5.895, -5.18 , -6.   , ..., -7.96 , -8.01 , -8.01 ],
       [-7.227, -5.824, -7.12 , ..., -7.77 , -7.766, -7.74 ],
       [-6.812, -5.73 , -6.984, ..., -7.84 , -7.816, -7.816]],
      dtype=float16), end_logits=array([[-5.367, -5.79 , -7.24 , ..., -8.16 , -8.14 , -8.12 ],
       [-5.6  , -5.254, -6.83 , ..., -8.15 , -8.11 , -8.12 ],
       [-6.37 , -4.715, -7.758, ..., -8.24 , -8.25 , -8.29 ],
       ...,
       [-3.973, -6.137, -5.758, ..., -8.086, -8.016, -8.02 ],
       [-6.01 , -5.715, -7.746, ..., -8.32 , -8.31 , -8.336],
       [-6.688, -6.46 , -7.145, ..., -8.11 , -8.11 , -8.13 ]],
      dtype=float16), hidden_states=None, attentions=None)

In [31]:
modelsko.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [32]:
modelsko_best_model_preds = modelsko.evaluate(tf_test_dataset)

1819/1819 [==============================] - 103s 54ms/step - loss: 2.0131 - end_logits_loss: 0.9623 - start_logits_loss: 1.0508 - end_logits_accuracy: 0.7309 - start_logits_accuracy: 0.6906


In [33]:
modelsko_best_model_preds

[2.0131354331970215,
 0.9623449444770813,
 1.050808310508728,
 0.7308961153030396,
 0.6906267404556274]

In [25]:
from transformers import pipeline

# Replace this with your own checkpoint
question_answerer = pipeline("question-answering", model="nlp-polish/squad-bert-1")

context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at nlp-polish/squad-bert-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


{'score': 0.984375,
 'start': 78,
 'end': 105,
 'answer': 'Jax, PyTorch and TensorFlow'}

In [26]:
model_utils.plot_and_save_fig_from_history(
    history,
    attributes=["accuracy", "val_accuracy"],
    title="Model accuracy",
    y_label="Accuracy",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{training_name}_accuracy.png",
)

model_utils.plot_and_save_fig_from_history(
    history,
    attributes=["loss", "val_loss"],
    title="Model loss",
    y_label="Loss",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{training_name}_loss.png",
)

NameError: name 'model_utils' is not defined

In [ ]:
best_model = model_utils.get_best_model_from_checkpoints(
    model,
    history,
    checkpoints_dir=checkpoints_dir,
    checkpoint_filename_template=checkpoint_filename_template,
)

In [ ]:
model_utils.save_model(
    best_model,
    model_name=model_name,
    training_number=training_number,
    saved_models_dir=constants.SAVED_MODEL_LOCATION,
    default_model_version=constants.DEFAULT_MODEL_VERSION,
)

In [ ]:
# Evaluation
best_model.evaluate(tf_test_dataset, batch_size=batch_size)

In [ ]:
class_preds = model_utils.get_class_preds(model, tf_test_dataset)

In [ ]:
precision, recall, f1 = model_utils.get_classification_evaluation_metrics(
    class_actual=tokenized_test_dataset["emotions"],
    class_preds=class_preds,
    average="micro",
)

print(f"Precision score: ", precision)
print(f"Recall score: ", recall)
print(f"F1 score: ", f1)

In [ ]:
model_utils.print_incorrectly_predicted_texts(
    texts=raw_dataset["text_pl"],
    class_actual=raw_dataset["emotions"],
    class_preds=class_preds,
)